# RMSNorm 均方根归一化
## 原理
RMS归一化的目标是将输入数据规范化到一个稳定的范围。它是通过除以输入数据的均方根（RMS）值来实现的。
与批量归一化（Batch Normalization）不同，RMS归一化没有依赖于批次的均值和方差，而是直接使用输入数据。

<img src="../images/RMSNorm.png" alt="RMSNorm公式" title="RMSNorm" width=600 height=300 />

具体计算步骤如下：  
① 对输入数据求平方。  
② 沿着指定的维度计算平均值。  
③ 计算均方根（即平方均值的平方根），然后取其倒数。  
④ 使用步骤3得到的归一化因子乘以输入数据。   

## 权重参数weight的作用
因为虽然归一化操作可以使得神经网络中不同层的输出值保持在同一数值范围内，提高训练的稳定性，但它同时也可能会<u>**破坏原有的数据分布**</u>， 影响模型的表达能力。通过引入可学习的权重参数，模型可以在训练过程中自动学习到一个合适的缩放系数，既保持了数据的归一化，又避免了破坏原有的数据分布，从而平衡了模型的稳定性和表达能力。   

<font color=red>权重参数的具体作用如下:</font>   
**(1) 恢复表示的能力：**  
归一化技术，如RMSNorm，将输入数据规范化到某个范围（通常接近0）。但在某些情况下，原始的尺度或某个不同的尺度可能是有益的或更适合某些任务的。权重参数允许模型学习恢复（或更改）这种尺度。   

**(2) 提供额外的训练信号：**
当权重参数与归一化技术一起使用时，模型可以学习如何最佳地缩放每个特征，这可以视为一个额外的训练信号。在某些情况下，这可能会提高模型的性能。   

**(3) 增加模型容量：**
添加权重参数增加了模型的参数数量，从而增加了模型的容量。虽然这可能会增加过拟合的风险，但与合适的正则化技术结合使用时，这可以帮助模型更好地拟合数据。        

**(4) 与其他归一化技术的一致性：**    
如前所述，许多其他归一化技术（例如Batch Normalization和Layer Normalization）都包含缩放和偏移参数。在RMSNorm中引入权重参数可以为研究者和开发者提供一个熟悉和一致的界面，使得从一个归一化技术切换到另一个技术变得更加容易。

In [ ]:
from torch import nn
import torch
class RMSNorm(nn.Module):
    def __init__(self, dim, eps=1e-8):
        super(RMSNorm, self).__init__()
        # 为了数值稳定性而添加的小常数
        self.eps = eps 
        # 可学习的权重参数，初始值为1，它的维度与输入特征的维度相同。
        self.weight = nn.Parameter(torch.ones(dim))
    
    def _norm(self, x):
        '''实现了均方根归一化的计算过程'''
        # 首先计算x沿着最后一个维度的平方均值，然后，使用torch.rsqrt来计算其平方根的倒数。
        # 然后，将这个归一化因子乘到x上。
        return x*torch.rsqrt(torch.mean(x**2, dim=-1, keepdim=True)+self.eps)
    
    def forward(self, x):
        '''重写父类的forward方法，定义模块的前向传播。'''
        # 对输入x进行归一化，这里使用了.type_as(x) 来确保归一化的结果和输入x有相同的数据类型
        output = self._norm(x.float()).type_as(x)
        # 将归一化后的输出乘以权重参数，得到最终的输出。
        return output * self.weight